In [0]:
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()

# secret key for database connection
# w.secrets.create_scope(scope="demo_secrets")
# w.secrets.put_secret( scope="demo_secrets",key="server-hostname",string_value="")
# w.secrets.put_secret (scope="demo_secrets",key="http-path",string_value="")
# w.secrets.put_secret(scope="demo_secrets",    key="databricks-token",string_value="")
secrets = w.secrets.list_secrets(scope="demo_secrets")
for secret in secrets:
    value = dbutils.secrets.get(
        scope="demo_secrets",
        key=secret.key
    )
    print(f"Key: {secret.key}, First 10 chars: {value[:10]}")

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS jp_assessment;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS jp_assessment.latam_lab_silver;
CREATE SCHEMA IF NOT EXISTS jp_assessment.latam_lab_gold;
CREATE SCHEMA IF NOT EXISTS jp_assessment.latam_lab_landing_zone;



In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.pipelines import FileLibrary
import json

def flatten_config(config):
    return {k: str(v) for k, v in config.items()}

with open(
    "/Workspace/Repos/ram/ai-ram-databricks/dlt_pipeline/gold_pipeline_settings.json",
    "r"
) as f:
    pipeline_settings = json.load(f)

w = WorkspaceClient()

pipeline = w.pipelines.create(
    name="gold-lakeflow-pipeline",
    development=True,
    libraries=[
        FileLibrary(
            path="/Workspace/Repos/ram/ai-ram-databricks/......./gold_pipelines.py"
        )
    ],
    configuration=flatten_config(pipeline_settings)
)

print(f"Pipeline created with ID: {pipeline.pipeline_id}")

In [0]:
%pip install databricks-sql-connector
dbutils.library.restartPython()



In [0]:
from databricks import sql

connection = sql.connect(
    server_hostname=dbutils.secrets.get(
        scope="demo_secrets",
        key="server-hostname"
    ),
    http_path=dbutils.secrets.get(
        scope="demo_secrets",
        key="http-path"
    ),
    access_token=dbutils.secrets.get(
        scope="demo_secrets",
        key="databricks-token"
    )
)

cursor = connection.cursor()

# Get all schemas
cursor.execute("SELECT schema_name FROM information_schema.schemata")
schemas = [row[0] for row in cursor.fetchall()]

# Get all tables with their schemas
cursor.execute(
    "SELECT table_schema, table_name FROM information_schema.tables"
)
tables = cursor.fetchall()

# Organize tables under their schemas
schema_tables = {schema: [] for schema in schemas}
for schema, table in tables:
    if schema in schema_tables:
        schema_tables[schema].append(table)

# Print in tree-like fashion
for schema, tables in schema_tables.items():
    print(f"{schema}/")
    for table in tables:
        print(f"  └─ {table}")

cursor.close()
connection.close()

In [0]:
from databricks import sql

connection = sql.connect(
    server_hostname=dbutils.secrets.get(
        scope="demo_secrets",
        key="server-hostname"
    ),
    http_path=dbutils.secrets.get(
        scope="demo_secrets",
        key="http-path"
    ),
    access_token=dbutils.secrets.get(
        scope="demo_secrets",
        key="databricks-token"
    )
)

cursor = connection.cursor()

cursor.execute("CREATE SCHEMA IF NOT EXISTS demo")
cursor.execute("""
    CREATE TABLE IF NOT EXISTS demo.demo_table (
        id INT,
        name STRING
    )
""")
cursor.execute("""
    INSERT INTO demo.demo_table
      VALUES (1, 'Alice'), (2, 'Bob'), (3, 'Carol')
""")

cursor.execute("SELECT * FROM demo.demo_table")
rows = cursor.fetchall()

print("Rows in demo.demo_table:")
for row in rows:
    print(row)

cursor.close()
connection.close()